In [ ]:
!pip install fasttext

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
import fasttext
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [ ]:
cb_df = pd.read_csv('/content/clickbait.csv')
ncb_df = pd.read_csv('/content/notClickbait.csv')

In [ ]:
print(cb_df.shape)
print(ncb_df.shape)

(101, 6)
(100, 6)


In [ ]:
cb_df['isClickbait'] = 1
ncb_df['isClickbait'] = 0

In [ ]:
cb_df.head()

,ID,Video Title,Views,Likes,Dislikes,Favorites,isClickbait
0,5WTXHdc1_zI,10 People You Don't Want To Mess With,484411,3881,191,0.0,1
1,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,24646,0.0,1
2,aKkRwPGSWGQ,10 Real Life Giants You Won't Believe Exist,3674544,12116,1570,0.0,1
3,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,2858,0.0,1
4,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1954,0.0,1


In [ ]:


ncb_df.head()

,ID,Video Title,Views,Likes,Dislikes,Favorites,isClickbait
0,uocETPj4Jx4,"Extreme $500,000 Game Of Tag!",28527480,1912728,14689,0,0
1,EgkUEKt_qmA,APEX LEGENDS MAKES NO SENSE 3,338383,23061,277,0,0
2,5jyj5aJaLfc,Minecraft But Blocks Randomly Change When You ...,1119179,69823,1678,0,0
3,5bQUqTsC0JA,I TOOK 10 PENALTIES AGAINST A PREMIER LEAGUE K...,1363546,98995,597,0,0
4,A-Uva3rlc8s,"Minecraft, But You Can Shear Any Block...",1671133,71767,1788,0,0


In [ ]:

df = pd.concat([cb_df, ncb_df], ignore_index=True)

In [ ]:

df.shape

(201, 7)

In [ ]:
df.head()

,ID,Video Title,Views,Likes,Dislikes,Favorites,isClickbait
0,5WTXHdc1_zI,10 People You Don't Want To Mess With,484411,3881,191,0.0,1
1,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,24646,0.0,1
2,aKkRwPGSWGQ,10 Real Life Giants You Won't Believe Exist,3674544,12116,1570,0.0,1
3,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,2858,0.0,1
4,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1954,0.0,1


In [ ]:
df.drop(['Dislikes', 'Favorites'], axis=1, inplace=True)

In [ ]:
df.head()

,ID,Video Title,Views,Likes,isClickbait
0,5WTXHdc1_zI,10 People You Don't Want To Mess With,484411,3881,1
1,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,1
2,aKkRwPGSWGQ,10 Real Life Giants You Won't Believe Exist,3674544,12116,1
3,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,1
4,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1


## Removing Duplicates

In [ ]:
duplicates = df[df.duplicated('ID', keep=False)]

In [ ]:
duplicates

,ID,Video Title,Views,Likes,isClickbait
16,zxYjTTXc-J8,"Last To Leave Circle Wins $500,000",21445488,1729921,1
26,ck1EQ-vmmc4,he fell off then died..,54361,4818,1
27,ck1EQ-vmmc4,he fell off then died..,54361,4818,1
82,yVaMtCi9Xw0,10 Cute Kids Who Aged Horribly,22943937,64582,1
84,yVaMtCi9Xw0,10 Cute Kids Who Aged Horribly,22943937,64582,1
176,zxYjTTXc-J8,"Last To Leave Circle Wins $500,000",40712459,2390269,0


In [ ]:

df.drop_duplicates('ID', keep='first', inplace=True)

In [ ]:

df.shape

(198, 5)

In [ ]:

df['isClickbait'].value_counts()

isClickbait
1    99
0    99
Name: count, dtype: int64

## Image Analysis

In [ ]:
!unzip /content/notClickbait.zip

In [ ]:
!unzip /content/isClickbait.zip

In [ ]:
# import shutil
# import os

# source_dir = '/content/isClickbait/notClickbait'
# target_dir = '/content/isClickbait'

# file_names = os.listdir(source_dir)

# for file_name in file_names:
#     target_file = os.path.join(target_dir, file_name)
#     if os.path.exists(target_file):
#         os.remove(target_file)

#     shutil.move(os.path.join(source_dir, file_name), target_dir)



In [ ]:
new_titles = { # run this to change the titles so it can capture the exact image path
    '92XN2kog_uI': 'GHOST HUNTING in GTA 5! (DO NOT Play at 3am)',
    'UE8lAZVIwac':	'DONT PLAY GTA 5 AT 3 AM.. (scary)'
}

for video_id, title in new_titles.items():
    df.loc[df['ID'] == video_id, 'Video Title'] = title


In [ ]:
image_folder = '/content/thumbnails'
files = os.listdir(image_folder)
data = []

for file in files:
    parts = file.split('___')
    video_id = parts[0]
    video_title = '___'.join(parts[1:]).rsplit('.', 1)[0]  # Remove the extension and rejoin title
    # print(video_title)
    path = os.path.join(image_folder, file)
    data.append({'ID': video_id, 'Video Title': video_title, 'image_path': path})

image_df = pd.DataFrame(data)

merged_df = pd.merge(df, image_df, on=['ID', 'Video Title'], how='left')

In [ ]:
index_to_drop = df[df['ID'] == "0uocETPj4Jx4"].index
# i found this to be the invalid video id

df.drop(index_to_drop, inplace=True)

In [ ]:
merged_df['image_path'].isna().value_counts()

image_path
False    197
Name: count, dtype: int64

I had to delete one row, because it had an invalid video ID.

In [ ]:
merged_df[merged_df['image_path'].isna()]

,ID,Video Title,Views,Likes,isClickbait,image_path


In [ ]:
merged_df.head(10)

,ID,Video Title,Views,Likes,isClickbait,image_path
0,5WTXHdc1_zI,10 People You Don't Want To Mess With,484411,3881,1,/content/thumbnails/5WTXHdc1_zI___10 People Yo...
1,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,1,/content/thumbnails/hxwpkM5w3Cc___I Got Hunted...
2,aKkRwPGSWGQ,10 Real Life Giants You Won't Believe Exist,3674544,12116,1,/content/thumbnails/aKkRwPGSWGQ___10 Real Life...
3,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,1,/content/thumbnails/81mXAkFEkMs___10 Real Life...
4,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1,/content/thumbnails/9NQQKNAwaPk___10 Mythical ...
5,Y_PQrPRolvM,Real Giants Caught On Camera!,518170,4033,1,/content/thumbnails/Y_PQrPRolvM___Real Giants ...
6,ZZtYAOpHVYk,10 Real Life GIANTS,9244943,64025,1,/content/thumbnails/ZZtYAOpHVYk___10 Real Life...
7,8Iuvf4_NMF4,15 Biggest Animals You Won't Believe Actually ...,8380807,48083,1,/content/thumbnails/8Iuvf4_NMF4___15 Biggest A...
8,n9Ch7nP7suA,I Went Over Speed House And My Brothers Pulled...,182683,18504,1,/content/thumbnails/n9Ch7nP7suA___I Went Over ...
9,T8Ffq1X1ygQ,Why Nobody Wants This Part of Africa,1142293,41508,1,/content/thumbnails/T8Ffq1X1ygQ___Why Nobody W...


## Data Processing and Word Embedding

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def preprocess(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    return tokens


In [ ]:
merged_df["processed_title"] = merged_df['Video Title'].apply(lambda x: preprocess(x))

In [ ]:
merged_df.head()

,ID,Video Title,Views,Likes,isClickbait,image_path,processed_title
0,5WTXHdc1_zI,10 People You Don't Want To Mess With,484411,3881,1,/content/thumbnails/5WTXHdc1_zI___10 People Yo...,"[10, peopl, you, do, n't, want, to, mess, with]"
1,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,1,/content/thumbnails/hxwpkM5w3Cc___I Got Hunted...,"[i, got, hunt, by, the, fbi]"
2,aKkRwPGSWGQ,10 Real Life Giants You Won't Believe Exist,3674544,12116,1,/content/thumbnails/aKkRwPGSWGQ___10 Real Life...,"[10, real, life, giant, you, wo, n't, believ, ..."
3,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,1,/content/thumbnails/81mXAkFEkMs___10 Real Life...,"[10, real, life, giant, you, wo, n't, believ, ..."
4,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1,/content/thumbnails/9NQQKNAwaPk___10 Mythical ...,"[10, mythic, creatur, that, actual, exist]"


In [ ]:
merged_df["processed_title"].to_csv("words_for_fasttext.txt", index=False, header=False)

In [ ]:
# import fasttext.util

# fasttext.util.download_model('en', if_exists='ignore')
# ft_en_model = fasttext.load_model('cc.en.300.bin')

# def normalize_vector(vec):
#     norm = np.sqrt(np.sum(vec**2))
#     if not norm==0:
#         return vec/norm
#     else:
#         return vec


In [ ]:
# model = fasttext.train_unsupervised("/content/words_for_fasttext.txt", model='skipgram', minCount=1)

# vector = model.get_word_vector('FBI')
# print(vector.shape)

In [ ]:

# def title_to_vector(title):
#     words = title.split()
#     vectors = [model.get_word_vector(word) for word in words if word in model.words]
#     return sum(vectors) / len(vectors) if vectors else np.zeros(model.get_dimension())

# df['title_vector'] = df['processed_title'].apply(title_to_vector)


In [ ]:
from gensim.models import Word2Vec #word2vec >>> fasttext and nltk in terms of performance

w2v_model = Word2Vec(sentences=merged_df['processed_title'], vector_size=100, window=5, min_count=1, workers=4, sg=1)

def title_to_vector(title):
    vectors = [w2v_model.wv[word] for word in title if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)


merged_df['title_vector'] = merged_df['processed_title'].apply(title_to_vector)

w2v_model.save("word2vec_model.model")

In [ ]:
merged_df.head()

,ID,Video Title,Views,Likes,isClickbait,image_path,processed_title,title_vector
0,5WTXHdc1_zI,10 People You Don't Want To Mess With,484411,3881,1,/content/thumbnails/5WTXHdc1_zI___10 People Yo...,"[10, peopl, you, do, n't, want, to, mess, with]","[0.0036908041, 0.0025779884, 0.002334288, -0.0..."
1,hxwpkM5w3Cc,I Got Hunted By The FBI,42724724,2005151,1,/content/thumbnails/hxwpkM5w3Cc___I Got Hunted...,"[i, got, hunt, by, the, fbi]","[0.0025038028, -0.0018399571, -0.0022768166, -..."
2,aKkRwPGSWGQ,10 Real Life Giants You Won't Believe Exist,3674544,12116,1,/content/thumbnails/aKkRwPGSWGQ___10 Real Life...,"[10, real, life, giant, you, wo, n't, believ, ...","[0.005157443, 0.002462668, 0.0027790754, 0.003..."
3,81mXAkFEkMs,10 Real Life Giants You Won't Believe Exist,6890718,15222,1,/content/thumbnails/81mXAkFEkMs___10 Real Life...,"[10, real, life, giant, you, wo, n't, believ, ...","[0.005157443, 0.002462668, 0.0027790754, 0.003..."
4,9NQQKNAwaPk,10 Mythical CREATURES That Actually Existed,2089601,46750,1,/content/thumbnails/9NQQKNAwaPk___10 Mythical ...,"[10, mythic, creatur, that, actual, exist]","[0.0020510824, 0.0019501689, 0.00090245187, 0...."


In [ ]:

merged_df = merged_df.sample(frac=1, random_state=42) # random shuffle rows
merged_df.head()

,ID,Video Title,Views,Likes,isClickbait,image_path,processed_title,title_vector
140,B6v6I4u7LJU,WHO CAN PUNCH THE HARDEST IN AMP,232821,14510,0,/content/thumbnails/B6v6I4u7LJU___WHO CAN PUNC...,"[who, can, punch, the, hardest, in, amp]","[0.00023425916, 0.00014988093, -0.0016295245, ..."
113,H5O5MEcqGBU,BABY # 2 NAME REVEAL! (SUPER EMOTIONAL AND CUTE!),213991,7882,0,/content/thumbnails/H5O5MEcqGBU___BABY # 2 NAM...,"[babi, #, 2, name, reveal, !, (, super, emot, ...","[-0.0032075658, 0.0032264679, -0.00025418034, ..."
16,zxYjTTXc-J8,"Last To Leave Circle Wins $500,000",21445488,1729921,1,/content/thumbnails/zxYjTTXc-J8___Last To Leav...,"[last, to, leav, circl, win, $, 500,000]","[0.0015838064, 0.0021177898, 9.079381e-06, 0.0..."
75,jPiPvAafhPU,LUCKY PEOPLE Who Avoided VERY CLOSE CALLS,13431007,136547,1,/content/thumbnails/jPiPvAafhPU___LUCKY PEOPLE...,"[lucki, peopl, who, avoid, veri, close, call]","[0.0021841438, -0.0010507915, 0.0018662575, 0...."
155,oqICzYBjjTw,DaBaby - Wockesha (Freestyle) [Official Video],1823136,137724,0,/content/thumbnails/oqICzYBjjTw___DaBaby - Woc...,"[dababi, -, wockesha, (, freestyl, ), [, offic...","[-0.0024978195, 0.0045816996, -0.0008538628, -..."


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197 entries, 140 to 102
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               197 non-null    object
 1   Video Title      197 non-null    object
 2   Views            197 non-null    int64 
 3   Likes            197 non-null    int64 
 4   isClickbait      197 non-null    int64 
 5   image_path       197 non-null    object
 6   processed_title  197 non-null    object
 7   title_vector     197 non-null    object
dtypes: int64(3), object(5)
memory usage: 13.9+ KB


## Finding frequent itemsets using Association rules

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

clickbait_titles = merged_df[merged_df['isClickbait'] == 1]['processed_title']


te = TransactionEncoder()
te_ary = te.fit(clickbait_titles).transform(clickbait_titles)
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)


print(df_onehot.head())

       !      $      &     'm     's    've      (      )      +  +more  ...  \
0  False   True  False  False  False  False  False  False  False  False  ...   
1  False  False  False  False  False  False  False  False  False  False  ...   
2  False  False  False  False  False  False  False  False  False  False  ...   
3   True  False  False  False  False  False  False  False  False  False  ...   
4  False  False  False  False  False  False  False  False  False  False  ...   

    work  worker  world  worst  would  wrong     xp    you   your  youtub  
0  False   False  False  False  False  False  False  False  False   False  
1  False   False  False  False  False  False  False  False  False   False  
2  False   False  False  False  False  False  False   True  False   False  
3  False   False  False  False  False  False  False  False  False   False  
4  False   False  False  False  False  False  False  False  False   False  

[5 rows x 397 columns]


In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df_onehot, min_support=0.05, use_colnames=True)

print(frequent_itemsets)

     support           itemsets
0   0.295918                (!)
1   0.244898                (()
2   0.244898                ())
3   0.091837                (,)
4   0.061224                (-)
..       ...                ...
62  0.051020         (in, (, ))
63  0.051020    (mrbeast, (, ))
64  0.061224     (youtub, (, ))
65  0.051020       (), (, ,, !)
66  0.051020  (), youtub, (, !)

[67 rows x 2 columns]


Through frequent itemset analysis, we could see that exclamation marks, parantheses were frequent among clickbaits.

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

rules = rules[rules['lift'] > 1.2]

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

       antecedents consequents   support  confidence      lift
0              (5)         (!)  0.051020    0.833333  2.816092
1            (the)         (!)  0.051020    0.714286  2.413793
2         (youtub)         (!)  0.051020    0.833333  2.816092
3              (()         ())  0.244898    1.000000  4.083333
4              ())         (()  0.244898    1.000000  4.083333
5              (,)         (()  0.061224    0.666667  2.722222
6              (5)         (()  0.051020    0.833333  3.402778
7             (in)         (()  0.051020    0.714286  2.916667
8        (mrbeast)         (()  0.051020    0.714286  2.916667
9         (youtub)         (()  0.061224    1.000000  4.083333
10             (,)         ())  0.061224    0.666667  2.722222
11             (5)         ())  0.051020    0.833333  3.402778
12            (in)         ())  0.051020    0.714286  2.916667
13       (mrbeast)         ())  0.051020    0.714286  2.916667
14        (youtub)         ())  0.061224    1.000000  4

## Preparing dataset for model training

In [ ]:
vectors = np.array(merged_df["title_vector"].tolist())

X = np.hstack((vectors, merged_df[["Views", "Likes"]].values))
y = merged_df["isClickbait"].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
param_grid = { #svm but trash performance
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1, 10, 100],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'degree': [2, 3, 4]
}


In [ ]:
# grid search
grid_search = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5, scoring='accuracy')

In [ ]:
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END .........C=0.001, degree=2, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .........C=0.001, degree=2, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .........C=0.001, degree=2, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .........C=0.001, degree=2, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .........C=0.001, degree=2, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END ........C=0.001, degree=2, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ........C=0.001, degree=2, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ........C=0.001, degree=2, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ........C=0.001, degree=2, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ........C=0.001, degree=2, gamma=scale, kernel=poly; total time=   0.0s
[CV] END .....C=0.001, degree=2, gamma=scale, kernel=sigmoid; total time=   0.0s
[CV] END .....C=0.001, degree=2, gamma=scale,

In [ ]:
print("Best model: ", grid_search.best_params_)
best_model = grid_search.best_estimator_


In [ ]:
predictions = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))


### Trying out AdaBoost & KNN for text data


In [ ]:
rf_grid = {
    'n_estimators': [50, 100, 200, 300, 500],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0],
    # Optionally, configure some parameters of the base estimator:
    # 'base_estimator__max_depth': [1, 2, 3]
}

In [ ]:
grid_search_rf = GridSearchCV(AdaBoostClassifier(), rf_grid, refit=True, verbose=2, cv=5, scoring='accuracy')


In [ ]:
grid_search_rf.fit(X_train, y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   0.2s
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   0.2s
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   0.2s
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   0.2s
[CV] END ...............learning_rate=0.001, n_estimators=50; total time=   0.2s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   0.3s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   0.3s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   0.3s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   0.3s
[CV] END ..............learning_rate=0.001, n_estimators=100; total time=   0.3s
[CV] END ..............learning_rate=0.001, n_estimators=200; total time=   0.6s
[CV] END ..............learning_rate=0.001, n_e

GridSearchCV(cv=5, estimator=AdaBoostClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0],
                         'n_estimators': [50, 100, 200, 300, 500]},
             scoring='accuracy', verbose=2)

In [ ]:
print("Best model: ", grid_search_rf.best_params_)
best_model = grid_search_rf.best_estimator_


Best model:  {'learning_rate': 0.01, 'n_estimators': 200}


In [ ]:
predictions = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.85
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        22
           1       0.83      0.83      0.83        18

    accuracy                           0.85        40
   macro avg       0.85      0.85      0.85        40
weighted avg       0.85      0.85      0.85        40



In [ ]:
# Specify the path to save the model
model_filename = 'trained_model_ada.pkl'

# Open a file with write-binary ('wb') permission and save the model
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### knn (similar performance)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
knn_grid = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}


knn = KNeighborsClassifier()


pipeline = Pipeline([
    ('knn', knn)
])


grid_search_knn = GridSearchCV(pipeline, knn_grid, cv=5, verbose=1, scoring='accuracy')

In [ ]:
grid_search_knn.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5, estimator=Pipeline(steps=[('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': [3, 5, 7, 9],
                         'knn__weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [ ]:
best_knn = grid_search_knn.best_estimator_
best_knn

Pipeline(steps=[('knn',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=9))])

In [ ]:
predictions = best_knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.825
              precision    recall  f1-score   support

           0       0.86      0.82      0.84        22
           1       0.79      0.83      0.81        18

    accuracy                           0.82        40
   macro avg       0.82      0.83      0.82        40
weighted avg       0.83      0.82      0.83        40



### Training a Bagging Classifier having the base estimator = Best KNN model

In [ ]:
bagging_knn = BaggingClassifier(
    base_estimator=best_knn,
    n_estimators=50,
    random_state=42,
    max_samples=1.0,
    max_features=1.0
)

# Fit Bagging
bagging_knn.fit(X_train, y_train)

# Predict and evaluate
y_pred_bagging = bagging_knn.predict(X_test)
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f'Accuracy with Bagging KNN: {accuracy_bagging}')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy with Bagging KNN: 0.825


## Using Deep Learning (ignore)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.5),
    LSTM(32, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 1/100
20/20 [==============================] - 6s 76ms/step - loss: 0.6970 - accuracy: 0.4777 - val_loss: 0.6950 - val_accuracy: 0.4250
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6848 - accuracy: 0.5669 - val_loss: 0.6917 - val_accuracy: 0.4250
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6758 - accuracy: 0.6497 - val_loss: 0.6883 - val_accuracy: 0.5000
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6574 - accuracy: 0.7452 - val_loss: 0.6840 - val_accuracy: 0.5500
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6384 - accuracy: 0.8025 - val_loss: 0.6779 - val_accuracy: 0.6000
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6054 - accuracy: 0.8599 - val_loss: 0.6711 - val_accuracy: 0.6000
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5819 - accuracy: 0.7898 - val_loss: 0.6614 - val_accuracy: 0.6000
Epoch 8/100


In [ ]:
y_test.size

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


40

In [ ]:
!zip "/content/thumbnails.zip" "/content/thumbnails/"

  adding: content/thumbnails/ (stored 0%)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
merged_df.to_csv("/content/merged_youtube_clickbait.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## CNN for image clickbait detection

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
merged_df.head()


,ID,Video Title,Views,Likes,isClickbait,image_path,processed_title,title_vector
140,B6v6I4u7LJU,WHO CAN PUNCH THE HARDEST IN AMP,232821,14510,0,/content/thumbnails/B6v6I4u7LJU___WHO CAN PUNC...,"[who, can, punch, the, hardest, in, amp]","[0.00023425916, 0.00014988093, -0.0016295245, ..."
113,H5O5MEcqGBU,BABY # 2 NAME REVEAL! (SUPER EMOTIONAL AND CUTE!),213991,7882,0,/content/thumbnails/H5O5MEcqGBU___BABY # 2 NAM...,"[babi, #, 2, name, reveal, !, (, super, emot, ...","[-0.0032075658, 0.0032264679, -0.00025418034, ..."
16,zxYjTTXc-J8,"Last To Leave Circle Wins $500,000",21445488,1729921,1,/content/thumbnails/zxYjTTXc-J8___Last To Leav...,"[last, to, leav, circl, win, $, 500,000]","[0.0015838064, 0.0021177898, 9.079381e-06, 0.0..."
75,jPiPvAafhPU,LUCKY PEOPLE Who Avoided VERY CLOSE CALLS,13431007,136547,1,/content/thumbnails/jPiPvAafhPU___LUCKY PEOPLE...,"[lucki, peopl, who, avoid, veri, close, call]","[0.0021841438, -0.0010507915, 0.0018662575, 0...."
155,oqICzYBjjTw,DaBaby - Wockesha (Freestyle) [Official Video],1823136,137724,0,/content/thumbnails/oqICzYBjjTw___DaBaby - Woc...,"[dababi, -, wockesha, (, freestyl, ), [, offic...","[-0.0024978195, 0.0045816996, -0.0008538628, -..."


In [ ]:
merged_df['isClickbait'] = merged_df['isClickbait'].map({0: 'no', 1: 'yes'})

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_df, temp_df = train_test_split(merged_df, test_size=0.4, random_state=42)


validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='isClickbait',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_df,
    x_col='image_path',
    y_col='isClickbait',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = validation_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='isClickbait',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 118 validated image filenames belonging to 2 classes.
Found 39 validated image filenames belonging to 2 classes.
Found 40 validated image filenames belonging to 2 classes.


In [ ]:
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
base_model = hub.KerasLayer(model_url, input_shape=IMAGE_SIZE + (3,), trainable=False)

# Build the cnn_model
cnn_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# from tensorflow.keras.layers import Input, Flatten, Dense
# from tensorflow.keras.models import Model

# base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))

# # Freeze the layers of the base model
# base_model.trainable = False

# x = Flatten(name='flatten')(base_model.output)
# x = Dense(128, activation='relu', name='fc1')(x)
# x = Dense(64, activation='relu', name='fc2')(x)
# x = Dense(16, activation='relu', name='fc3')(x)
# output = Dense(1, activation='sigmoid', name='predictions')(x)

# # Combine the base model and the custom layers into a new model
# cnn_model = Model(inputs=base_model.input, outputs=output)


In [ ]:
cnn_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Suitable for binary classification
    metrics=['accuracy']
)

cnn_model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_22 (KerasLayer  (None, 1280)              2257984   
 )                                                               
                                                                 
 dense_90 (Dense)            (None, 128)               163968    
                                                                 
 dense_91 (Dense)            (None, 64)                8256      
                                                                 
 dense_92 (Dense)            (None, 16)                1040      
                                                                 
 dense_93 (Dense)            (None, 1)                 17        
                                                                 
Total params: 2431265 (9.27 MB)
Trainable params: 173281 (676.88 KB)
Non-trainable params: 2257984 (8.61 MB)
__________

In [ ]:
import warnings # removing too many warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [ ]:
history = cnn_model.fit(
    train_generator,
    epochs=70,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 1/70
4/4 [==============================] - ETA: 0s - loss: 0.3785 - accuracy: 0.8220
Epoch 1: val_accuracy improved from -inf to 0.64103, saving model to best_model.h5
4/4 [==============================] - 7s 2s/step - loss: 0.3785 - accuracy: 0.8220 - val_loss: 0.6186 - val_accuracy: 0.6410
Epoch 2/70
4/4 [==============================] - ETA: 0s - loss: 0.3596 - accuracy: 0.8305
Epoch 2: val_accuracy improved from 0.64103 to 0.66667, saving model to best_model.h5
4/4 [==============================] - 7s 1s/step - loss: 0.3596 - accuracy: 0.8305 - val_loss: 0.7129 - val_accuracy: 0.6667
Epoch 3/70
4/4 [==============================] - ETA: 0s - loss: 0.3618 - accuracy: 0.8390
Epoch 3: val_accuracy improved from 0.66667 to 0.71795, saving model to best_model.h5
4/4 [==============================] - 6s 1s/step - loss: 0.3618 - accuracy: 0.8390 - val_loss: 0.6116 - val_accuracy: 0.7179
Epoch 4/70
4/4 [==============================] - ETA: 0s - loss: 0.2790 - accuracy: 0.8729

In [ ]:
intest_loss, test_accuracy = cnn_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")
test_loss

2/2 [==============================] - 2s 183ms/step - loss: 0.7047 - accuracy: 0.6750
Test Accuracy: 0.675000011920929


1.7568919658660889

In [ ]:
merged_df.head()

,ID,Video Title,Views,Likes,isClickbait,image_path,processed_title,title_vector
140,B6v6I4u7LJU,WHO CAN PUNCH THE HARDEST IN AMP,232821,14510,NaN,/content/thumbnails/B6v6I4u7LJU___WHO CAN PUNC...,"[who, can, punch, the, hardest, in, amp]","[0.00023425916, 0.00014988093, -0.0016295245, ..."
113,H5O5MEcqGBU,BABY # 2 NAME REVEAL! (SUPER EMOTIONAL AND CUTE!),213991,7882,NaN,/content/thumbnails/H5O5MEcqGBU___BABY # 2 NAM...,"[babi, #, 2, name, reveal, !, (, super, emot, ...","[-0.0032075658, 0.0032264679, -0.00025418034, ..."
16,zxYjTTXc-J8,"Last To Leave Circle Wins $500,000",21445488,1729921,NaN,/content/thumbnails/zxYjTTXc-J8___Last To Leav...,"[last, to, leav, circl, win, $, 500,000]","[0.0015838064, 0.0021177898, 9.079381e-06, 0.0..."
75,jPiPvAafhPU,LUCKY PEOPLE Who Avoided VERY CLOSE CALLS,13431007,136547,NaN,/content/thumbnails/jPiPvAafhPU___LUCKY PEOPLE...,"[lucki, peopl, who, avoid, veri, close, call]","[0.0021841438, -0.0010507915, 0.0018662575, 0...."
155,oqICzYBjjTw,DaBaby - Wockesha (Freestyle) [Official Video],1823136,137724,NaN,/content/thumbnails/oqICzYBjjTw___DaBaby - Woc...,"[dababi, -, wockesha, (, freestyl, ), [, offic...","[-0.0024978195, 0.0045816996, -0.0008538628, -..."


In [ ]:
!zip "/content/thumbnails.zip" "/content/thumbnails/"

  adding: content/thumbnails/ (stored 0%)


In [ ]:
merged_df.to_csv("/content/merged_youtube_clickbait.csv")

## SVM for image classification (ignore)

In [ ]:
def extract_features(image_paths):

    images = [tf.keras.preprocessing.image.load_img(path, target_size=(224, 224)) for path in image_paths]
    images_array = np.array([tf.keras.preprocessing.image.img_to_array(img) for img in images])
    preprocessed_images = tf.keras.applications.vgg16.preprocess_input(images_array)


    features = base_model.predict(preprocessed_images)


    flattened_features = features.reshape(features.shape[0], -1)

    return flattened_features

In [ ]:
image_paths = merged_df['image_path'].tolist()
features = extract_features(image_paths)
labels = merged_df['isClickbait'].to_numpy()

7/7 [==============================] - 100s 14s/step


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


svm_model_img = make_pipeline(StandardScaler(), SVC(kernel='poly', C=1e5, probability=True))
svm_model_img.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=100000.0, kernel='poly', probability=True))])

In [ ]:
predictions = svm_model_img.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.6
              precision    recall  f1-score   support

          no       0.58      1.00      0.73        22
         yes       1.00      0.11      0.20        18

    accuracy                           0.60        40
   macro avg       0.79      0.56      0.47        40
weighted avg       0.77      0.60      0.49        40

